## 1. Business Understanding

**The Overview**

In this table dataset pre-processing and visualization lab, I will use the dataset that's from kaggle competition: Zillow’s Home Value Prediction. Zillow's Zestimates are the estimate of home values based on 7.5 million statistical and machine learning models that analyze hundreds of data points on each property. More details about zillow estimate's introduction can be found: https://www.zillow.com/zestimate/ . The final result and purpose of this dataset is the accurate prediction of log(Zillow estimate propeties' price)-log(actual transaction price). In this lab, I just focused on the preprocessing ,visulization of variables' relationships and dimensional reduction as preparation of following models'construction. The possible models to predict the logerror are Multi-Layer Perceptron , Convolutional Networks and Recurrent Networks

#### The Bussiness Purposes:
The accurate estimate of the logerror between zillow website's price and actual transaction price is very important. 
why? we can get the reasons from following cases:
1. For Zillow website, if pepole can figure out a model, based on CNN or RNN, the Zillow website can modify their large error variable in their computational function by checking the weight of different variables. The Zillow website price function can be found here: https://www.zillow.com/research/zestimate-forecast-methodology/ . Moreover, the Zillow website can find a better way to model the estimate function after discovering this data for this data covers a lot of features related to properties. The Zillow webstite can delete duplicated or add features in Zestimate function in the process of data mining and machine learning. 
2. For some house sellers and investors, houses' price prediction is important. A better estimate of house price will help them to avoid loss of money in the properties' transaction. Usually, before investors make the decision to sell or buy a house, they will take the important estimate websites' results as criterion. Too big logerror will actually led to large amount loss.  Considering a house which can be sold at price 100,0000 after 6 monthes, if the Zestimate has 0.1 logerror under estimate, the seller will lost 20,0000 if they sell the house at the estimated price now.
3. For families, they will be more careful about specific type of properties' estimate accuracy. Usually, they are just focused on houses or apartments that are suitable for family living. So the improvement of these kind of properties' price estimation will make more people get benefit from this website.
4. For government officers, a better estimate of propeties' transaction price in different time will benefit them to calculate the properties of person or companies and also will be helpful for officers to forcast the financial development in different places and make more financial plans based on these predictional results.


**The Steps for Lab one:**
1. Analyze the data features' properties based on the information provied in the website. 
2. Analyze the data's quality based on the missing, duplicated values information.
3. Analyze the disctribution of important variables.
4. Discover relationships between important variables and do the dimension reduction to get most important variables.


**The expectant result: How well the algorithm to perform to be useful:**

Based on the information provied on the Zillow website: https://www.zillow.com/zestimate/#acc , we can know that the distribution of logerror is like bellowing: 

WITHIN 5% OF SALE PRICE:
This is the percentage of transactions in a location for which the Zestimate was within 5% of the transaction price. Nationwide, Zestimates are currently within 5% of the final sale price 54.4% of the time.

WITHIN 20% OF SALE PRICE:
This is the percentage of transactions in a location for which the Zestimate was within 20% of the transaction price. Nationally, Zestimates are currently within 20% of the final sale price 86.9% of the time.

So, if our target is to improve the estimate then the estimate of more than 86.9% properties have less than 20% error will be considered as better estimate. But for this problem, the comptetion holder want people to get accurate estimate of the error of Zestimate. That means the smaller difference of log(zillow estimate)-log(actual price) and my error estimate, the better result it will be. 

## 2. Data Understanding 

#### First Let's import data from dataset to python, which contains two tables: 
#### The dataset can be found in website: https://www.kaggle.com/c/zillow-prize-1/data  please download it to see the iploty pictures in jupyter notebook.
 1) train_2016 (90275 rows × 3 columns): the result of log(zillowestimate price)-log(actual transaction price) in 2016 

 2) properties_2016 (2985217 rows × 57features): table the features'data of different properties. some features are highly related and some are useless for our analysis. I will first make groups based on their definition,then do the distribution and relationship analysis.

In [ ]:
# import data from kaggle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dicom
import os
import seaborn as sns
from operator import itemgetter
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
import warnings
import plotly 
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
plotly.tools.set_credentials_file(username='ZhangAnyu', api_key='CogwNQlPuCThZLOjAOZQ')
warnings.filterwarnings('ignore')
train_2016=pd.read_csv('/Users/MacBook/Desktop/machinlearning/lab1/train_2016_v2.csv', parse_dates=["transactiondate"])
properties_2016=pd.read_csv('/Users/MacBook/Desktop/machinlearning/lab1/properties_2016.csv')

### 1) table of logerror in 2016

In [ ]:
train_2016.head()

This table contains three types of data: 

1) parcelid (integer): the ID number of each properties

2) logerror (float): log(Zillow estimate price)-log(actual transaction price)

3) transectiondata (object): the year-month-date of actual transaction


### Table logerrror: Data Quality 

Data Quality check is an important step considering the missing values' impact to classification and prediction analysis. And we need to make sure our preprocessed data have unique houseID to avoid the mistake in mapping process with properties table. The properties table has unique Parcelid.

### 1) Missing Values

In [ ]:
train_2016.isnull().sum()

There is no missing values after checking the missing values in logerror table in 2016.

### 2)Duplicate houses ID


In [ ]:
duplicatedtrain=train_2016[train_2016.duplicated(['parcelid'], keep=False)]
duplicatedtrain.head()

### Why Duplicated data appear? 

Actually the Duplicated PracelID in this table dose not mean poor data quality, it means same house has more than one transaction in 2016. From the table we can know that for parcelid 13850164, it has two transcations in 2016. 

Next, let's count how many transactions each duplicated ID have.

In [ ]:
print(duplicatedtrain['parcelid'].value_counts().describe())
print((duplicatedtrain['parcelid'].value_counts()==3).sum())
print((duplicatedtrain['parcelid'].value_counts()==2).sum())
print(duplicatedtrain['parcelid'].value_counts().head())
doubleduplicated=duplicatedtrain[~(duplicatedtrain['parcelid']==11842707)]
doubleduplicated1=pd.DataFrame(doubleduplicated['logerror'])
errorchange=[]
print (doubleduplicated1.head()) 

So,we can know that there are 124 houses have duplicated transection information which means same house was selled more than one time in 2016. There is only one house had three trasection history in 2016 and all of the other have twice transections in 2016. In the visualization section, I will do the statistical analysis of these duplicated selled houses and do the logerror analysis based on their properties like latitude and longitude in the later labs. And i will not delete these duplicated ID.

### Distribution of Logerror

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
print(train_2016.logerror.describe())
x = train_2016.logerror
data = [go.Histogram(x=x)]
iplot(data, filename='basic histogram of logerror')

Basd on the statistic data, we can know that logerror data don't have the problem of outlier. The logerror distribution graph is close to normal distribution. The existence of large logerror makes us hard to know the most of logerror. So, I deleted the large logerrors which distance to mean are more than two stdandard deviations (SD).

In [ ]:
import plotly
import math
print (plotly.__version__)     
# version 1.9.x required
plotly.offline
from plotly.offline import iplot_mpl
print(np.mean(train_2016.logerror))
print(np.std(train_2016.logerror))
def removeoutlier(data):
    mean = np.mean(data)
    std = np.std(data)
    leave = [i for i in data if (mean - 2 * std < i < mean + 2 * std)]
    return leave
filtered_d = removeoutlier(train_2016.logerror)
fig = plt.figure()
sns.set_palette("hls")
sns.distplot(filtered_d);
iplot_mpl(fig,strip_style=False)


Much Better! this ditribution is close to normal distribution with mean=0.011457219606756682 and standard deviation(SD)= 0.16107794320832886. For finding the relationship between logerror and properties, i do the feature discretization to logerror in the next step.

      abs(logerror) < abs(Mean-SD） => 'low error'

      abs(logerror) >= abs(Mean-SD) => 'high error'

   ### Feature Discretization: Logerror

In [ ]:
meansd=abs(-0.16107794320832886/2+0.011457219606756682) # use this based on exprience to make lowerror number and high error number dont have huge difference.
train_2016['class'] = pd.cut(abs(train_2016.logerror),[0,meansd,1e6],3,
                                 labels=['lowerror','higherror']) # this creates a new variable
train_2016.head()

### 2) Table of  Properties' Features

In [ ]:
properties_2016.head()

This is the table of first 5 instances in the file of houses' properties. Then let's check the data type of features and do the data quality analysis. I noticed there are a lot Null values in the table. 

In [ ]:
print(properties_2016.info())

In [ ]:
properties_2016.describe()

The describe() function gives us the distribution of all non object features. There are a lot of null values in some features. Some features are highly related and some features are useless for furture analysis of logerror.  

### 1) Missing Values

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot_mpl, iplot
%matplotlib inline 
#matplotlib.style.use('ggplot')
null_count=pd.DataFrame(properties_2016.isnull().sum())
null_count.columns=['nullnumber']
sortednull=null_count.sort_values(by='nullnumber')
data = [go.Bar(
            x=sortednull.index,
            y=sortednull.nullnumber
    )]

iplot(data, filename='basic-bar')


In [ ]:
print(properties_2016.shape)
nullpercent=sortednull
nullpercent.columns=['Null percent']
print(nullpercent[:31]/2985217)

we know that properties_2016 table dimension is 2985217 * 58. we totally have 58 features. If a feature has more than 2000000 null, it means this feature has more than more than 60% data are null. In this dataset, we have 29 features have more than 60% null data. So we can first consider 29 features that has less than 60% null data. In these 29 features, 25 features have less than 10% null data. Let's first consider these features' relationship and importance. Here, parcelID is the nominal feature which should not be considered relationship discover.


### 2) Duplicated Meaning Features

By checking the meaning of these 30 features, we found that feature: rawcensustruactandblock  and censustractandblock have same meaning. Let's figure out if they can be represented by one of them.
First, let's check the distribution of them.

In [ ]:
properties_2016[['rawcensustractandblock','censustractandblock']].describe()

Second, let's check if they have linear relationship by visulization.

In [ ]:
plt.scatter(properties_2016['rawcensustractandblock'], properties_2016['censustractandblock'])
plt.show()

These two variables have same meaning and are discrete. so we can pick one of them in further analysis. Here, i picked rawcensustractandblock
in further relationship analysis.

### 3）Duplicated ParcelID 


In [ ]:
properties_2016[properties_2016.duplicated(['parcelid'], keep=False)]

No duplicated ParcelID found in properties_2016 table.

# 3. Data Visulization 
## 1.Make subgroups based on feature meaning


Considering we still have 31 features left which less than 60% null data. 
we first made the subgroups of features based on their meaning in table Zillow_data_dictionary. Bellowing are features that can be grouped as fllowing:




### subgroup I :  Location of Properties:
#### rawcensustractandblock, regionidcounty, longitude, latitude, regionidzip , regionidcity, regionidneighborhood.

### subgroup II : Structrue and Condition of Properties:
####  'parcelid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt','buildingqualitytypeid','heatingorsystemtypeid','logerror'

### subgroup III: Time:

#### 'yearbuilt','assessmentyear', ' transaction month'
                       
                                         
### subgroup IV: Value and Area  of Proeperties and land:

#### 'calculatedfinishedsquarefeet','finishedsquarefeet12','taxamount','taxvaluedollarcnt','structuretaxvaluedollarcnt'








## 2. Relationships of Subgroups and Logerror

### I. Location of Parcel VS logerror

In [ ]:
properties_2016[['rawcensustractandblock','regionidcounty','longitude','latitude','regionidzip','regionidcity','regionidneighborhood']].describe()

In [ ]:
from sklearn import preprocessing
from pandas.tools.plotting import scatter_matrix
#mergelogerror_properties = pd.merge(properties_2016,train_2016, on='parcelid')
%matplotlib inline 
locationproperties=mergelogerror_properties[['longitude','latitude','regionidzip','regionidcity','regionidneighborhood','class']].sample(5000)
locationproperties['longitude']=locationproperties.longitude/1e6
locationproperties['latitude']=locationproperties.latitude/1e6
locationproperties1=locationproperties.dropna()
sns.pairplot(locationproperties1,diag_kind="kde",hue="class",markers='+',plot_kws=dict(s=1,edgecolor="g", linewidth=1),diag_kws=dict(shade=True))

From the pairplot, we know that the city area which are in the middle or close to los angles has more chance to have high error. And the regionidzip in the range [96200-96500] have more chance to get low error.

In [ ]:
# plot the correlation matrix using seaborn
%matplotlib inline 
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.set(style="darkgrid") # one of the many styles to plot using
f, ax = plt.subplots(figsize=(7, 7))
sns.heatmap(locationproperties.corr(), cmap=cmap, annot=True)
f.tight_layout()

we can say two variable have correlationship if correlationship number is in the range : r<-0.3 or r>0.3 more details can be found in this website: http://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/

So, based on the calculation results we can know that latitude has correlationship with regionidneighborhood, longitude, regionidcounty and longtitude has correlationship with rawcensustractandblock, regionidcounty and latitude. Otherwise, based the meaning of these location features, we can know that longtitude and latitude will be good represents for location subgroups.

### Properties' Density Distrbution

In [ ]:
y=mergelogerror_properties.latitude.values/1e6
x=mergelogerror_properties.longitude.values/1e6
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)# Calculate the point density
fig, ax = plt.subplots()
ax.scatter(x, y, c=z, s=100, edgecolor='')
ax.set_xlabel('longtitude')
ax.set_ylabel('latitude')
plt.title('Properties Density Distrbution')
plt.show()

### Properties' Distribution in different cities by Google Map 

In [ ]:
%matplotlib inline 
import webbrowser
url = "file://" + '/Users/MacBook/Desktop/machinlearning/lab1' + "/" + "visual.html"
webbrowser.open(url)

For the dataset is huge for google map plot, i choose 1% of longtitude and latitude data: 29738 points to show the distribution of properties in different cities. For the html file is now located on my computer and can not be opened outside my computer, I put the picture of result of distribution of properties in different cities.

In [ ]:
from IPython.display import Image
Image("/Users/MacBook/Desktop/machinlearning/lab1/distributionmap.png")

The number inside the circle means how many properies are in this area. From the picture we can know that los angles has most of properties and the Sana Ana area and long beach area are in second place. This result is not a big surprise but still gives us a more straightforward sense of properties' distribution. Next, let's check the distribution of logerror in different location(different longitude and altitude).

In [ ]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Marker, Layout, XAxis, YAxis
sample1=mergelogerror_properties.sample(5000)
lowerror1 = sample1[(sample1['class'] =='lowerror')]
higherror1 = sample1[(sample1['class']=='higherror')]

trace_comp0 = go.Scatter(
    y=lowerror1.latitude/1e6,
    x=lowerror1.longitude/1e6,
    mode='markers',
    marker=Marker(size=lowerror1.logerror, sizemode='area', sizeref=0.01,color='navy'),
    name='lowerror',
    text=lowerror1.logerror,
    )

trace_comp1 = go.Scatter(
    y=higherror1.latitude/1e6,
    x=higherror1.longitude/1e6,
    mode='markers',
    marker=Marker(size=higherror1.logerror, sizemode='area', sizeref=0.01,color='red'),
    name='higherror',
    text=higherror1.logerror,
        )

data_comp = [trace_comp0, trace_comp1]
layout_comp = go.Layout(
    title='Location VS logerror',
    hovermode='closest',
    xaxis=dict(
        title='longitude)',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis=dict(
        title='latitude',
        ticklen=5,
        gridwidth=2,
    ),
)
fig_comp = go.Figure(data=data_comp, layout=layout_comp)
py.iplot(fig_comp, filename='location vs logerror')




This picture also shows that high error properties are more dense around los angles. And the biggest high error is in the location longitude: [-118.5,-118] and latitude: [34.2,33.8].

### II. Structrue and Condition of Parcel VS Logerror

In [ ]:
#from matplotlib import scatter
structioncondition=mergelogerror_properties[['parcelid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt','buildingqualitytypeid','heatingorsystemtypeid','logerror','class']]
structioncondition['lowerror']=[0 if i!='lowerror' else 1 for i in structioncondition['class']]

#print(structioncondition['class'])
errorgroup = pd.crosstab([structioncondition['heatingorsystemtypeid'],
                    structioncondition['buildingqualitytypeid']], 
                    structioncondition['class'])
print(errorgroup)
#errorgroup.plot(kind='bar', stacked=True)
error_rate = errorgroup.div(errorgroup.sum(1).astype(float),
                             axis=0) # normalize the value

# print survival_rate
error_rate.plot(kind='barh', 
                   stacked=True)
plt.title('lowerror and high error percentage by heating system type and building quality type')
plt.show()

I made crosstab by class of heatingorsystemtypeid  and buildingqualitytypeid, here properties mainly have three types of heating systems(2: Central, 7: floor/wall and 20: Solar) and building quality id is from best (1) to worst (12). Here, True represents Low error and False represents High error.

From the bar plot, we can know that over 75% properties which have central heating system and 4 buliding quality have low error. And central heating houses with worst buliding quality only have about 58% low error percent. it means the accuracy will be bad when the buliding quality is poor.
Although for the floor/wall heating houses and solar heating houses, if they have poor buliding quality like 10, their low error percent will be less than 70%.



In [ ]:
structionconditionnumber=structioncondition[['buildingqualitytypeid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt']]
structionconditionnumber.describe()

In [ ]:
%matplotlib inline 
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.set(style="darkgrid") # one of the many styles to plot using
f, ax = plt.subplots(figsize=(9, 9))
sns.heatmap(structioncondition[['bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt','buildingqualitytypeid','heatingorsystemtypeid','logerror']].corr(), cmap=cmap, annot=True)
f.tight_layout()

Based on the calculation results we can know that bedroom number, bathroom number , full bathroom number and calculated bath room number are highly related with correlation number more than 0.6. So, we can choose one variable to represent the imformation of these 4. Here, i choose the bathroom number. I also noticed that heating system type has a high positive relationship with room number.

### III Logerror VS Time

### Logerror VS Yearbuilt and Assessment year 

In [ ]:
timeproperties=mergelogerror_properties[['yearbuilt','assessmentyear','logerror','class']]
timeproperties.describe()

The properties table contains houses with built year from 1885 to 2015. they all have the same assessmentyear. So, the assessment year feature is useless in the future. Next, let's check how built year is related to logerror.

In [ ]:
# built year distribution
timeproperties=timeproperties[['yearbuilt','logerror','class']]
sns.distplot(properties_2016.yearbuilt[~np.isnan(properties_2016.yearbuilt)])
plt.title('Distribution of parcels with builtyear ')
plt.show()

We can know most of properties were built in 1950 to 1990.


In [ ]:
#ax = scatter_matrix(timeproperties,figsize=(15, 10))
logerrortimemean=timeproperties[['logerror','yearbuilt']].groupby('yearbuilt'). mean()
plotly.offline.init_notebook_mode() # run at the start of every notebook
plotly.offline.iplot({
    "data": [{
        "x": logerrortimemean.index,
        "y": logerrortimemean.logerror
    }],
    "layout": {
        "title": "Mean of logerror VS Built year"
    }
})


From the mean of logerror Vs built year picture, we can know that with the time chage, mean of logerror is getting smaller and more stable.

### Transaction Month VS Logerror

In [ ]:
#plot the hist of transection month to see the transaction tendency.
import matplotlib.pyplot as plt
from scipy.stats import norm
train_2016['month']=train_2016['transactiondate'].dt.month
plt.hist(train_2016['month'],bins=12)
plt.title('hist of number of transactions in every month in 2016 ')
plt.show()

There are less transaction after 10.25 for the transaction data are treated as test data. In the month 1-10, month in the middle of year have tendency to have most transactions like June. 

In [ ]:
# show the mean of logerror for every month
trainlogerrormean=train_2016[['logerror','month']].groupby('month'). mean()
plotly.offline.init_notebook_mode() # run at the start of every notebook
plotly.offline.iplot({
    "data": [{
        "x": [1,2,3,4,5,6,7,8,9,10,11,12],
        "y": trainlogerrormean.logerror
    }],
    "layout": {
        "title": "Mean of logerror VS Transaction Month"
    }
})

which is a suprise that, with more transaction in the middle of year, the mean of logerror is much smaller than other months. There are many reasons, one is for the transaction number is more than other months, June have more low error than other months.

In [ ]:
# show the mean of abs(logerror) for every month
abstrain_2016=train_2016
abstrain_2016['logerror']=abs(train_2016['logerror'])
abstrain_2016mean=abstrain_2016[['logerror','month']].groupby('month'). mean()
plotly.offline.init_notebook_mode() # run at the start of every notebook
plotly.offline.iplot({
    "data": [{
        "x": [1,2,3,4,5,6,7,8,9,10,11,12],
        "y": abstrain_2016mean.logerror
    }],
    "layout": {
        "title": "Mean of Abs(logerror) VS Transaction Month"
    }
})


But if we plot the mean of abs(logerror), we can find that error decrease with transaction month. I think this is beacuase pepole start to collect information of properties in the begining of the year. To the end of the year people will avoid the error based on the tendency of previous months.

In [ ]:
mergelogerror_properties['month']=mergelogerror_properties['transactiondate'].dt.month
sns.violinplot(x="month", y="yearbuilt", hue="class", data=mergelogerror_properties,
               split=True, inner="quart")
plt.title('Violion plot of logerror by built year and transaction month')
plt.show()

By violin picture, we can know most of high error properties are built in older time than lowerror. The mean of built year of high error is about 1958 and the mean of built year of low error is about 1970. There is no big difference of low error and high error in each month.

### IV. Area and Value of Property and Land

In [ ]:
areapropertyland=mergelogerror_properties[['parcelid','calculatedfinishedsquarefeet','finishedsquarefeet12','lotsizesquarefeet','taxamount','taxvaluedollarcnt','structuretaxvaluedollarcnt','propertycountylandusecode','propertyzoningdesc','landtaxvaluedollarcnt','logerror','class']]
areapropertyland.describe()
#sns.pairplot(areapropertyland, hue="class", size=2)
%matplotlib inline 
cmap = sns.diverging_palette(220, 10, as_cmap=True) # one of the many color mappings
sns.set(style="darkgrid") # one of the many styles to plot using
f, ax = plt.subplots(figsize=(9, 9))
sns.heatmap(areapropertyland[['calculatedfinishedsquarefeet','finishedsquarefeet12','lotsizesquarefeet','taxamount','taxvaluedollarcnt','structuretaxvaluedollarcnt','propertycountylandusecode','propertyzoningdesc','landtaxvaluedollarcnt','logerror']].corr(), cmap=cmap, annot=True)
plt.title('Correlation Matrix Graph')
f.tight_layout()

calculatedfinishedsquarefeet','finishedsquarefeet12','taxamount','taxvaluedollarcnt','structuretaxvaluedollarcnt','propertycountylandusecode','propertyzoningdesc','landtaxvaluedollarcnt' are all high related. so we can choose one of them and lotsizesquarefeet to represent this group. 

## 4. Exceptional Work

### House Properties Dimension Reduction

### 1)PCA of Main Features

Based on the analysis above, I picked out most important features: longitude, latitude, built year, heating system type, building quality, taxt amount, properties area.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition.pca import PCA
from sklearn import preprocessing
df=mergelogerror_properties[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror']].copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet']]

print(df_scaled.head())
y = df.logerror
target_names='logerror'


 # fit data and then transform it
for k in range(1,6):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    print('explained_variance_ratio:',pca.explained_variance_ratio_)
    print('pca:',pca.components_)



In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.tools as tls
from sklearn.preprocessing import StandardScaler
X=df_scaled
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
u,s,v = np.linalg.svd(X_std.T)
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
# Make a list of (eigenvalue, eigenvector) tuples

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)
trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

#fig = Figure(data=data1, layout=layout)
py.iplot(Figure(data=data1, layout=layout))

Based on the cumulative explained variance we can know we need more than 4 principal component to get more than 70% cumulative explained variance. The reason is maybe we put too many variables as input. Let'delete some features like longitude , latitude, built year and calculate bath room number. 

In [ ]:
X=df_scaled[['fullbathcnt','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
u,s,v = np.linalg.svd(X_std.T)
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
# Make a list of (eigenvalue, eigenvector) tuples

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)
trace1 = Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

#fig = Figure(data=data1, layout=layout)
py.iplot(Figure(data=data1, layout=layout))

Still no big change. Anyway, let's still check the 2D picture with logerror class.

In [ ]:
#this function is from given lab one example
#print(df_scaled.head())
meansd=abs(-0.16107794320832886/2+0.011457219606756682)
train_2016['plotclass'] = pd.cut(abs(train_2016.logerror),[0,meansd,1e6],3,
                                 labels=['0','1']) # this creates a new variable
mergelogerror_properties = pd.merge(properties_2016,train_2016, on='parcelid')
df=mergelogerror_properties[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror','plotclass']].sample(1000).copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
#print(df.plotclass.describe())
y=df.plotclass

target_names='logerror'
 # fit data and then transform it
for k in range(1,6):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    #print('explained_variance_ratio:',pca.explained_variance_ratio_)
    #print('pca:',pca.components_)


from pandas.tools.plotting import scatter_plot
def get_feature_names_from_weights(weights, names):
    tmp_array = []
    for comp in weights:
        tmp_string = ''
        for fidx,f in enumerate(names):
            if fidx>0 and comp[fidx]>=0:
                tmp_string+='+'
            tmp_string += '%.2f*%s ' % (comp[fidx],f[:-5])
        tmp_array.append(tmp_string)
    return tmp_array
  
pca_weight_strings = get_feature_names_from_weights(pca.components_, df_scaled.columns) 
df_pca = pd.DataFrame(X_pca,columns=[pca_weight_strings])
y=preprocessing.scale(y)
ax = scatter_plot(df_pca, pca_weight_strings[0], pca_weight_strings[1], c=y,s=(y+2)*10)

From the 2D PCA of 1000 properties sample, we can know that there is a possible cluster in the middle for white points. And we also can know the weight of differnt variables, here full bathroom number and calculated bath room number has biggest weight , the heating system type is in second place. Actually low error and high error are not clustered enough for us to cluster methods further. Next, let's check if LDA has the same results.

### 2) Dimension Reduction of buliding construction by PCA

From the variables relationship with logerror, we know that room number seems most important. so, let's check if building construction variables can be represented by 2 PCs.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition.pca import PCA
from sklearn import preprocessing
df=mergelogerror_properties[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt','plotclass']].copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt']]

print(df_scaled.head())
y = df.plotclass
target_names='plotclass'

for k in range(1,3):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    print('explained_variance_ratio:',pca.explained_variance_ratio_)
    print('pca:',pca.components_)

from pandas.tools.plotting import scatter_plot
def get_feature_names_from_weights(weights, names):
    tmp_array = []
    for comp in weights:
        tmp_string = ''
        for fidx,f in enumerate(names):
            if fidx>0 and comp[fidx]>=0:
                tmp_string+='+'
            tmp_string += '%.2f*%s ' % (comp[fidx],f[:-5])
        tmp_array.append(tmp_string)
    return tmp_array
  
pca_weight_strings = get_feature_names_from_weights(pca.components_, df_scaled.columns) 
df_pca = pd.DataFrame(X_pca,columns=[pca_weight_strings])
y=preprocessing.scale(y)
ax = scatter_plot(df_pca, pca_weight_strings[0], pca_weight_strings[1], c=y,s=(y+2)*10)


At last, I found the 2 possible PCA components for number of rooms in houses. they have explained_variance_ratio: [ 0.70227567  0.19548756]. which means these two components can explain most of information in these five variables.   And consider the weight of each variables, we can know bathroomnumber, fullbathroomnumber and calculatedbathroomnumber most important variables in these room's number dataset considering the x direction component. If we consider the y direction component, then unit number will be the most important variable.

# Reference


https://pythonprogramming.net/matplotlib-3d-scatterplot-tutorial/

https://plot.ly/ipython-notebooks/principal-component-analysis/#pca-vs-lda

https://seaborn.pydata.org/tutorial/distributions.html

import plotly
print (plotly.__version__)
plotly.offline.init_notebook_mode() # run at the start of every notebook
from plotly.graph_objs import Scatter, Marker, Layout, XAxis, YAxis
mergelogerror_properties = pd.merge(properties_2016,train_2016, on='parcelid')
mergelogerror_properties1=mergelogerror_properties.sample(5000)
plotly.offline.iplot({
    'data':[
        Scatter(y=mergelogerror_properties1.latitude.values/1e6,
                x=mergelogerror_properties1.longitude.values/1e6,
                text=mergelogerror_properties1.logerror.astype(str),
                marker=Marker(size=mergelogerror_properties1.logerror, sizemode='area', sizeref=0.01,),
                mode='markers')
            ],
    'layout': Layout(xaxis=XAxis(title='longitude'), yaxis=YAxis(title='latitude'))
}, show_link=False)


train_2016['positive or negtive']=train_2016['transactiondate'].dt.month
for i in range(len(train_2016)):
    if train_2016['logerror'][i]>0:
        train_2016['positive or negtive'][i]='positive'
    else:
        train_2016['positive or negtive'][i]='negtive'
print(train_2016)
ax = sns.violinplot(x="month", y="logerror", hue="positive or negtive",data=tips, palette="muted", split=True)

from matplotlib.colors import ListedColormap
def plot_decision_regions(X, y, classifier, resolution=0.02):
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],alpha=0.8, c=cmap(idx),marker=markers[idx], label=cl)
from sklearn.lda import LDA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
lda=LDA(n_components=2)
sc=StandardScaler()
X_train,X_test,y_train,y_test=train_test_split(df_scaled,df.plotclass,test_size=0.2,random_state=0)
X_train_std=sc.fit_transform(X_train)
X_train_lda=lda.fit_transform(X_train_std,y_train)
lr=LogisticRegression()
lr=lr.fit(X_train_lda,y_train)
print(lr)
print(lr.class_weight)